#bike


In [83]:

import tensorflow as tf

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras import optimizers
from pandas import Series
import math
import numpy
from tensorflow.keras.layers import *
from tensorflow.keras import layers
import pandas as pd

SMALL_SIZE = 20
MEDIUM_SIZE = 23
BIGGER_SIZE = 25

plt.rc('font', size=SMALL_SIZE) # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE) # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE) # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE) # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE) # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE) # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) # fontsize of the figure title

data = pd.read_csv(os.path.realpath('./SeoulBikeData (1).csv'))

In [84]:
data['Holiday']=data['Holiday'].factorize()[0]

In [85]:
data['Seasons']=data['Seasons'].factorize()[0]
data['Functioning Day']=data['Functioning Day'].factorize()[0]

In [86]:
data['Rented Bike Count']

0        254
1        204
2        173
3        107
4         78
        ... 
8755    1003
8756     764
8757     694
8758     712
8759     584
Name: Rented Bike Count, Length: 8760, dtype: int64

In [87]:
data=data.drop(['Hour'],axis=1)

In [88]:
data.columns[1:-3]

Index(['Rented Bike Count', 'Temperature', 'Humidity(%)', 'Wind speed (m/s)',
       'Visibility (10m)', 'Dew point temperature', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)'],
      dtype='object')

In [89]:
cols=data.columns[1:-3] 
len(cols)

9

In [90]:
from sklearn.preprocessing import MinMaxScaler
values = data[cols].values.reshape(-1,9)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled=scaler.fit_transform(values)
#scaled = (values-(np.min(values)))/(np.max(values)-(np.min(values)))
#scaled = (values - np.mean(values))/np.std(values) 

In [91]:
np.max(values)

3556.0

In [92]:
np.min(values)

-30.6

In [93]:
np.max(values)-np.min(values)

3586.6

In [94]:
'''
from sklearn.preprocessing import MinMaxScaler
values = data[cols].values.reshape(-1,9)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values) 
'''

"\nfrom sklearn.preprocessing import MinMaxScaler\nvalues = data[cols].values.reshape(-1,9)\nvalues = values.astype('float32')\nscaler = MinMaxScaler(feature_range=(0, 1))\nscaled = scaler.fit_transform(values) \n"

In [95]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
    one_day = 7 * 24
    for row in range(values.shape[0]):
        for col in range(values.shape[1]):
            if values[row,0]==0:
                values[row,0] = values[row - one_day,0]

In [96]:
fill_missing(values)

In [97]:
sum(pd.DataFrame(values)[0]==0)

0

In [98]:

train_size = int(len(scaled) * 0.80)
val_size = int(len(scaled) * 0.10)

test_size = len(scaled) - train_size-val_size
train,val, test = scaled[0:train_size,:],scaled[train_size:train_size+val_size,:], scaled[train_size+val_size:,:]
print(len(train),len(val), len(test))

7008 876 876


In [99]:

def create_dataset_multistep(dataset, look_back=1,predict=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back-predict):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i+ look_back-1: i+ look_back+predict-1, 0])
    print(len(dataY))
    return np.array(dataX), np.array(dataY)

In [100]:
look_back = 168
predict=24

trainX, trainY = create_dataset_multistep(train, look_back,predict)
valX, valY = create_dataset_multistep(val, look_back,predict)
testX, testY = create_dataset_multistep(test, look_back,predict)

6816
684
684


In [101]:
testX.shape

(684, 168, 9)

In [103]:
def gradient_importance(seq, model):

    seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(seq)

    grads = tape.gradient(predictions, seq)
    grads = tf.reduce_mean(grads, axis=1).numpy()[0]
    
    return grads

def gradient_weight(seq, model):

    seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(seq)

    grad = tape.gradient(predictions, seq)
    #gr=grads
    #grads = tf.reduce_mean(grads, axis=1).numpy()[0]
    
    return grad


def activation_grad(seq, model):
    
    seq = seq[np.newaxis,:,:]
    grad_model = Model([model.inputs], 
                       [model.get_layer('extractor').output, 
                        model.output])

    # Obtain the predicted value and the intermediate filters
    with tf.GradientTape() as tape:
        seq_outputs, predictions = grad_model(seq)

    # Extract filters and gradients
    output = seq_outputs[0]
    grads = tape.gradient(predictions, seq_outputs)[0]

    # Average gradients spatially
    weights = tf.reduce_mean(grads, axis=0)
    
    # Get a ponderated map of filters according to grad importance
    cam = np.ones(output.shape[0], dtype=np.float32)
    for index, w in enumerate(weights):
        cam += w * output[:, index]

    time = int(seq.shape[1]/output.shape[0])
    cam = zoom(cam.numpy(), time, order=1)
    heatmap = (cam - cam.min())/(cam.max() - cam.min())
    
    return heatmap

## integrated_gradient_importance

In [31]:
def get_integrated_gradients_m(img_input, top_pred_idx, baseline2=None, num_steps=50):
    seq_input = X_test[1]
    
    mean = np.nanmean(seq_input)
    baseline2 = np.full(seq_input.shape, mean)
    # 1. Do interpolation.
    seq_input = seq_input.astype(np.float32)
    interpolated_seq = [
        baseline2 + (step / num_steps) * (seq_input - baseline2)
        for step in range(num_steps + 1)
    ]

    # 3. Get the gradients
    grads = []
    for i, seq in enumerate(interpolated_seq):
        seq = tf.expand_dims(seq, axis=0)
        with tf.GradientTape() as tape:
            tape.watch(seq)
            preds = model(seq)
        grad = tape.gradient(preds, seq)
        grads.append(grad[0])
    grads = tf.convert_to_tensor(grads, dtype=tf.float32)

    # 4. Approximate the integral using the trapezoidal rule
    grads = (grads[:-1] + grads[1:]) / 2.0
    avg_grads = tf.reduce_mean(grads, axis=0)

    # 5. Calculate integrated gradients and return
    integrated_grads = (seq_input - baseline2) * avg_grads
    return integrated_grads

## gradient_importance

In [102]:
def gradient_importance(seq, model):

    seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(seq)

    grads = tape.gradient(predictions, seq)
    grads = tf.reduce_mean(grads, axis=1).numpy()[0]
    
    return grads



def activation_grad(seq, model):
    
    seq = seq[np.newaxis,:,:]
    grad_model = Model([model.inputs], 
                       [model.get_layer('extractor').output, 
                        model.output])

    # Obtain the predicted value and the intermediate filters
    with tf.GradientTape() as tape:
        seq_outputs, predictions = grad_model(seq)

    # Extract filters and gradients
    output = seq_outputs[0]
    grads = tape.gradient(predictions, seq_outputs)[0]

    # Average gradients spatially
    weights = tf.reduce_mean(grads, axis=0)
    
    # Get a ponderated map of filters according to grad importance
    cam = np.ones(output.shape[0], dtype=np.float32)
    for index, w in enumerate(weights):
        cam += w * output[:, index]

    time = int(seq.shape[1]/output.shape[0])
    cam = zoom(cam.numpy(), time, order=1)
    heatmap = (cam - cam.min())/(cam.max() - cam.min())
    
    return heatmap

In [33]:
class CharacterTable(object):
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [34]:
from tensorflow.keras.models import Model

mtex

In [113]:
first_input = Input(shape=(168,9,1))

in0=Conv2D(filters=16, kernel_size=(168//3,1),input_shape=(168,9,1), activation='relu',padding='same')(first_input)
#in1= BatchNormalization(scale=False)(in1)

in1=Conv2D(filters=32, kernel_size=(168//5,1), activation='relu',padding='same')(in0)
#in0= BatchNormalization(scale=False)(in0)
#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)

in1=Conv2D(filters=1, kernel_size=1, activation='relu',padding='same')(in1)
#in1= BatchNormalization(scale=False)(in1)
print(in1.shape)
in1 = Reshape((168,9))(in1)
in0=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same',name='extractor')(in1)
#in1= BatchNormalization(scale=False)(in1)

#in1=Conv1D(filters=64, kernel_size=2, activation='relu',padding='same')(in0)
#in0= BatchNormalization(scale=False)(in0)
#in1=Conv1D(filters=64, kernel_size=3, activation='relu',padding='same')(in0)

#in1=Conv1D(filters=64, kernel_size=1, activation='relu',padding='same',name='extractor')(in1)
#in1=add([in0,in1],name='extractor')
#attn_layer = AttentionLayer(name='attention_layer')
#attn_out, attn_states = attn_layer([in1,in1])

    # Concat attention input and LSTM output, in original code it was decoder LSTM
#concat_out = Concatenate(axis=-1, name='concat_layer')([in1, attn_out])
#in1=MaxPooling1D(pool_size=2,name='extractor')(in1)

#in1=add([in0,in1])
in1=Flatten()(in0)
#in1=Dense(50, activation='relu')(in1)
#in1=LSTM(64)(in1)
in1 = Dense(32,activation='relu')(in1)

out=Dense(24)(in1)
model=tf.keras.Model(inputs=[first_input],outputs=[out])
model.compile(optimizer='adam', loss='mse')


checkpoint_path = "saved_weights/bike_MTEX_multistep(4).hdf5"
cb = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                        monitor='val_loss',mode='min',
                                                 verbose=1)
model.summary()
# fit model9
# history=model.fit(trainX, trainY, epochs=200, batch_size=100, validation_data=(valX, valY), verbose=1, callbacks=[cb])

(None, 168, 9, 1)
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 168, 9, 1)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 168, 9, 16)        912       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 168, 9, 32)        16928     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 168, 9, 1)         33        
_________________________________________________________________
reshape_3 (Reshape)          (None, 168, 9)            0         
_________________________________________________________________
extractor (Conv1D)           (None, 168, 64)           1792      
_________________________________________________________________
flatten_3 (Flatten)          (None, 

In [49]:
history=model.fit(trainX, trainY, epochs=200, batch_size=100, validation_data=(valX, valY), verbose=1, callbacks=[cb])

Epoch 1/200
69/69 [==============================] - ETA: 0s - loss: 0.0266
Epoch 00001: val_loss improved from inf to 0.03237, saving model to saved_weights\bike_MTEX_multistep(4).hdf5
69/69 [==============================] - 31s 452ms/step - loss: 0.0266 - val_loss: 0.0324
Epoch 2/200
69/69 [==============================] - ETA: 0s - loss: 0.0131
Epoch 00002: val_loss improved from 0.03237 to 0.02809, saving model to saved_weights\bike_MTEX_multistep(4).hdf5
69/69 [==============================] - 32s 467ms/step - loss: 0.0131 - val_loss: 0.0281
Epoch 3/200
69/69 [==============================] - ETA: 0s - loss: 0.0098
Epoch 00003: val_loss improved from 0.02809 to 0.02683, saving model to saved_weights\bike_MTEX_multistep(4).hdf5
69/69 [==============================] - 30s 433ms/step - loss: 0.0098 - val_loss: 0.0268
Epoch 4/200
69/69 [==============================] - ETA: 0s - loss: 0.0078
Epoch 00004: val_loss improved from 0.02683 to 0.02662, saving model to saved_weights\bi

KeyboardInterrupt: 

In [114]:
model.load_weights("saved_weights/bike_MTEX_multistep(4).hdf5")

In [51]:
model.inputs

[<tf.Tensor 'input_2:0' shape=(None, 168, 9, 1) dtype=float32>]

## integrated_gradient_importance

In [104]:
testX.shape

(684, 168, 9)

In [107]:
testX[id_].shape

(168, 9)

In [108]:
id_=(24)*24+12
import matplotlib.pyplot as plt
import numpy as np



grad_weight = get_integrated_gradients_m(testX[id_], model, baseline2 = X_test[id_-24*7])

grad_weight_abs = abs(grad_weight)

grad_weight_abs_z = (grad_weight_abs-(np.min(grad_weight_abs)))/(np.max(grad_weight_abs)-(np.min(grad_weight_abs)))
#grad_weight=grad_weight.reshape(48,4)

print(grad_weight_abs_z.shape)
plt.figure(figsize=(6,20))

#plt.plot(X_test[8], 'k')
a=grad_weight_abs_z.numpy()
a=a.reshape(168,9)
a=a.transpose()

#print(a)
fig, ax = plt.subplots(1,1, figsize=(15,5))

extent = [0 , 168, 0 , 9]
y_label_list = [1,3,5,7,9]
x_label_list = [0,24,48,72,96,120,144,168]
ax.set_xticks([0,24,48,72,96,120,144,168])

ax.set_yticks([10,30,50,70,90])

ax.set_xticklabels(x_label_list)
ax.set_yticklabels(y_label_list)
ax.set_ylabel('Feature')
ax.set_xlabel('Time (Hours)')
plt.subplots_adjust(bottom=0.15)


plt.imshow(a,extent = [0 , 168, 0 , 90], vmin=0, vmax=1, cmap="Blues", interpolation='nearest')
plt.colorbar()

IndexError: index 420 is out of bounds for axis 0 with size 1

## gradient_importance

In [55]:
X_test[12].shape

(168, 9)

In [80]:
X_test.shape[1]

168

In [82]:
X_test.shape

(1, 168, 9)

In [115]:
id_=(24)*24+12
import matplotlib.pyplot as plt
import numpy as np


X_test = testX.copy()

grad_weight = activation_grad(X_test, model)

grad_weight_abs = abs(grad_weight)

grad_weight_abs_z = (grad_weight_abs-(np.min(grad_weight_abs)))/(np.max(grad_weight_abs)-(np.min(grad_weight_abs)))
#grad_weight=grad_weight.reshape(48,4)

print(grad_weight_abs_z.shape)
plt.figure(figsize=(6,20))

#plt.plot(X_test[8], 'k')
a=grad_weight_abs_z.numpy()
a=a.reshape(168,9)
a=a.transpose()

#print(a)
fig, ax = plt.subplots(1,1, figsize=(15,5))

extent = [0 , 168, 0 , 9]
y_label_list = [1,3,5,7,9]
x_label_list = [0,24,48,72,96,120,144,168]
ax.set_xticks([0,24,48,72,96,120,144,168])

ax.set_yticks([10,30,50,70,90])

ax.set_xticklabels(x_label_list)
ax.set_yticklabels(y_label_list)
ax.set_ylabel('Feature')
ax.set_xlabel('Time (Hours)')
plt.subplots_adjust(bottom=0.15)


plt.imshow(a,extent = [0 , 168, 0 , 90], vmin=0, vmax=1, cmap="Blues", interpolation='nearest')
plt.colorbar()

UnimplementedError: The Conv2D op currently does not support grouped convolutions on the CPU. A grouped convolution was attempted to be run because the input depth of 9 does not match the filter input depth of 1 [Op:Conv2D]

In [111]:
import tensorflow as tf
weights = tf.ones([3,3,1,1])
X = tf.ones([1,32,32,1])
Y = tf.nn.conv2d(X, weights, [1, 1, 1, 1], "VALID")
print(Y.shape)
print(tf.__version__)

(1, 30, 30, 1)
2.3.0
